In [13]:
# Constants
OMEGA_BARYON = 0.046026

# file parts here
param_part = '../../param/file_parts/param.part'
doit_part = '../../param/file_parts/doit.part'
hydro_part = '../../param/file_parts/hydro.part'
SIDM_part = '../../param/file_parts/SIDM.part'

In [14]:
# Simulation Parameters
# Change these

N_part = 256
boxsize = 3 # in Mpc
code_directory = 1
mem_size = 8 # in GB

DM_type = '2cDM'
hydro_type = 'DM'
cross_section = 'power00_sigma1'
extra_info = 'var_2'
directory = f'dir_{code_directory}'

In [15]:
with open(param_part, 'r') as f:
    param_text = f.read()

with open(doit_part, 'r') as f:
    doit_text = f.read()

job_string = DM_type + f'_L{boxsize}N{N_part}_' + hydro_type + '_' + cross_section + '_' + extra_info + '_' + directory
boxsize_kpc = boxsize * 1000
softening_length = boxsize_kpc / (29 * N_part )
mem_size_MB = mem_size * 1000 

# replacing text in parameter file
param_text = param_text.replace('{$MEMSIZE}', str(mem_size_MB))
param_text = param_text.replace('{$CODE_DIR}', str(code_directory))
param_text = param_text.replace('{$BOXSIZE}', str(boxsize_kpc))

param_text = param_text.replace('{$SOFTENING_LENGTH}', str(softening_length))
param_text = param_text.replace('{$SOFTENING_LENGTH_HALF}', str(softening_length / 2)) 

if DM_type != 'CDM':
    with open(SIDM_part, 'r') as f:
        param_text += f.read()

if hydro_type == 'HY':
    hydro_flag = 1
    omega_baryon = OMEGA_BARYON
    with open(hydro_part, 'r') as f:
        param_text += f.read()
else:
    hydro_flag = 0
    omega_baryon = 0
param_text = param_text.replace('{$OMEGA_BARYON}', str(omega_baryon))
param_text = param_text.replace('{$HY_FLAG}', str(hydro_flag))

# replacing text in doit script
doit_text = doit_text.replace('{__MEMSIZE__}', f'{mem_size}g')
doit_text = doit_text.replace('{__JOBSTRING__}', job_string)
doit_text = doit_text.replace('{__CODE_DIR__}', str(code_directory))


param_name = job_string
param_out = '../../param/arepo/' + param_name
doit_text = doit_text.replace('{__PARAM_FILE__}', param_name)
doit_name = 'doit.sh-' + param_name
doit_out = '../../sh/' + doit_name

with open(param_out, 'w') as f:
    f.write(param_text)
with open(doit_out, 'w') as f:
    f.write(doit_text)